In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather

In [2]:
import glob
import os

In [3]:
#path for 11 models
path = "/home/mlopez/EXEC/tmax_annual_85/"

In [4]:
files = list(glob.glob(os.path.join(path,'*.*')))

In [5]:
files

['/home/mlopez/EXEC/tmax_annual_85/BNU-ESM_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/INM-CM4_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/CMCC-CMS_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/IPSL-CM5A-LR_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/IPSL-CM5B-LR_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/NorESM1-M_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/CanESM2_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/GFDL-ESM2M_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/HadGEM2-CC_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/MPI-ESM-LR_rcp85_tx_mean_annual.nc',
 '/home/mlopez/EXEC/tmax_annual_85/ACCESS1-3_rcp85_tx_mean_annual.nc']

# Create DF of all models

In [6]:
dflist = []

In [7]:
for file in files:
    df = pcdf.load_as_df(file)
    dflist.append(df)

/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between

In [8]:
dflist

[                                    tx_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  261.465546
                      1951-01-01  260.108063
                      1952-01-01  262.473969
                      1953-01-01  262.028931
                      1954-01-01  260.705750
 ...                                     ...
 40.041039 -74.045807 2096-01-01  298.027527
                      2097-01-01  296.727692
                      2098-01-01  296.084808
                      2099-01-01  296.549194
                      2100-01-01  296.858734
 
 [11507106 rows x 1 columns],                                     tx_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  262.232147
                      1951-01-01  261.559753
                      1952-01-01  262.615112
                      1953-01-01  262.848358
                      1954-01-01  264.110535
 ...                                     ...
 40.041039 -74.045807 20

In [9]:
#Takes 3 min
%time df11 = pd.concat(dflist)

CPU times: user 4min 1s, sys: 12min 46s, total: 16min 47s
Wall time: 1min 33s


In [10]:
df11

tx_mean
lat       lon        time                  
66.623306 -89.045212 1950-01-01  261.465546
                     1951-01-01  260.108063
                     1952-01-01  262.473969
                     1953-01-01  262.028931
                     1954-01-01  260.705750
...                                     ...
40.041039 -74.045807 2096-01-01  295.437378
                     2097-01-01  294.912170
                     2098-01-01  296.226624
                     2099-01-01  295.801758
                     2100-01-01  296.634033

[126578166 rows x 1 columns]

In [11]:
dftgm = df11.reset_index()

In [12]:
dftgm

,lat,lon,time,tx_mean
0,66.623306,-89.045212,1950-01-01,261.465546
1,66.623306,-89.045212,1951-01-01,260.108063
2,66.623306,-89.045212,1952-01-01,262.473969
3,66.623306,-89.045212,1953-01-01,262.028931
4,66.623306,-89.045212,1954-01-01,260.705750
...,...,...,...,...
126578161,40.041039,-74.045807,2096-01-01,295.437378
126578162,40.041039,-74.045807,2097-01-01,294.912170
126578163,40.041039,-74.045807,2098-01-01,296.226624
126578164,40.041039,-74.045807,2099-01-01,295.801758


# Get the mean for 11 models

In [13]:
#Takes 48s
%time dftgm2 = dftgm.groupby(["lat", "lon", "time"]).mean()

CPU times: user 1min 5s, sys: 4min 44s, total: 5min 49s
Wall time: 1min 28s


In [14]:
dftgm2

tx_mean
lat       lon        time                  
40.041039 -89.045212 1950-01-01  290.375061
                     1951-01-01  290.152832
                     1952-01-01  290.015076
                     1953-01-01  289.997925
                     1954-01-01  290.083313
...                                     ...
66.623306 -61.296322 2096-01-01  273.377197
                     2097-01-01  273.641754
                     2098-01-01  273.779999
                     2099-01-01  273.490784
                     2100-01-01  274.054962

[11507106 rows x 1 columns]

# Transform temp to Celsius

In [15]:
dfC = dftgm2.copy()
dfC["tx_mean"] = dfC["tx_mean"] -273.15
    

# Define 3 periods of time: 

In [16]:
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
#Get the mean of every period
dfC2 = dfC.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [17]:
dfp.reset_index()

,time,lat,lon,tx_mean
0,0.0,40.041039,-89.045212,17.477125
1,0.0,40.041039,-88.961884,17.465921
2,0.0,40.041039,-88.878548,17.450895
3,0.0,40.041039,-88.795219,17.370993
4,0.0,40.041039,-88.711891,17.391817
...,...,...,...,...
228613,2.0,66.623306,-61.629639,-1.113606
228614,2.0,66.623306,-61.546310,-1.500951
228615,2.0,66.623306,-61.462978,-1.256843
228616,2.0,66.623306,-61.379650,-1.183377


# Historic Period

485,881/3 = 161,960. ---- instead 333,469 rows × 4 columns

In [18]:
df_h = dfp.query("time==0.0")
df_h.reset_index()

,time,lat,lon,tx_mean
0,0.0,40.041039,-89.045212,17.477125
1,0.0,40.041039,-88.961884,17.465921
2,0.0,40.041039,-88.878548,17.450895
3,0.0,40.041039,-88.795219,17.370993
4,0.0,40.041039,-88.711891,17.391817
...,...,...,...,...
76201,0.0,66.623306,-61.629639,-6.145473
76202,0.0,66.623306,-61.546310,-6.505658
76203,0.0,66.623306,-61.462978,-6.248456
76204,0.0,66.623306,-61.379650,-6.158257


In [19]:
df2050 = dfp.query("time==1.0")
df2050.reset_index()

,time,lat,lon,tx_mean
0,1.0,40.041039,-89.045212,20.469864
1,1.0,40.041039,-88.961884,20.459538
2,1.0,40.041039,-88.878548,20.445108
3,1.0,40.041039,-88.795219,20.365299
4,1.0,40.041039,-88.711891,20.386372
...,...,...,...,...
76201,1.0,66.623306,-61.629639,-3.295894
76202,1.0,66.623306,-61.546310,-3.674468
76203,1.0,66.623306,-61.462978,-3.423926
76204,1.0,66.623306,-61.379650,-3.342954


In [20]:
df2080 = dfp.query("time==2.0")
df2080.reset_index()

,time,lat,lon,tx_mean
0,2.0,40.041039,-89.045212,22.461182
1,2.0,40.041039,-88.961884,22.451063
2,2.0,40.041039,-88.878548,22.436647
3,2.0,40.041039,-88.795219,22.357300
4,2.0,40.041039,-88.711891,22.378923
...,...,...,...,...
76201,2.0,66.623306,-61.629639,-1.113606
76202,2.0,66.623306,-61.546310,-1.500951
76203,2.0,66.623306,-61.462978,-1.256843
76204,2.0,66.623306,-61.379650,-1.183377


## Import dataframe with polygons

In [21]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

/home/mlopez/EXEC/anaconda3/envs/python37-2/lib/python3.8/site-packages/geofeather/core.py:85: UserWarning: Grid-TerritoiresGuides.feather.crs coordinate reference system file is missing. No crs will be set for this GeoDataFrame.
  warnings.warn(


In [22]:
dfpolyshape

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."
...,...,...,...,...
33193,45.041668,-72.208336,2c,"POLYGON ((-72.25000 45.00450, -72.25000 45.083..."
33194,45.041668,-72.125000,2c,"POLYGON ((-72.16666 45.00508, -72.16666 45.083..."
33195,45.041668,-72.041672,2c,"POLYGON ((-72.08334 45.00564, -72.08334 45.083..."
33196,45.041668,-71.958336,2c,"POLYGON ((-72.00000 45.00667, -72.00000 45.083..."


## Merge data with mask 

In [23]:
dftg11TG = pd.merge(df_h, dfpolyshape, on=["lat","lon"])

In [24]:
dftg11TG

,lat,lon,tx_mean,TER_GUIDE,geometry
0,44.957512,-74.629120,12.084303,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,12.088412,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,12.091355,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,12.102978,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,12.086818,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...
16563,52.873859,-67.629402,1.891784,6opqr,"POLYGON ((-67.67107 52.83219, -67.67107 52.915..."
16564,52.873859,-67.546066,1.950061,6opqr,"POLYGON ((-67.58773 52.83219, -67.58773 52.904..."
16565,52.957191,-67.712730,1.713015,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
16566,52.957191,-67.629402,1.822872,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."


In [ ]:
dftg11TG.TER_GUIDE.value_counts()

In [ ]:
TG4a = dftg11TG[dftg11TG.TER_GUIDE == "4a"]
TG4a

In [ ]:
TG4a.to_csv("/home/mlopez/EXEC/Tg_csv/TG4a11.csv")

# ---- Create GeoJson file ----

In [ ]:
geometry = dftg11TG["geometry"]

In [ ]:
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg11TG, crs=crs, geometry=geometry)

In [ ]:
gdf.to_file("TG11tg.json", driver="GeoJSON")